<a href="https://colab.research.google.com/github/harshj3915/Intuitionistic-FCM-Smote/blob/Harsh/Harsh_FCM-CSMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy
%pip install -q numpy scikit-learn imbalanced-learn
%pip install ucimlrepo matplotlib

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
print(iris.metadata) 
  
# variable information 
print(iris.variables) 


In [31]:
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt

def extract_and_visualize_k_clusters_with_dbscan(X, k):
    """
    Extract and visualize the first k clusters from the data using the DBSCAN algorithm.
    
    Args:
    - X : Feature matrix (dataset)
    - k : Number of clusters to extract
    
    Returns:
    - k_clusters : List of the first k clusters
    """
    # Apply DBSCAN to find clusters
    dbscan = DBSCAN()
    dbscan.fit(X)
    
    # Get the cluster labels assigned by DBSCAN
    cluster_labels = dbscan.labels_
    
    # Identify the unique cluster labels (ignoring noise, labeled as -1)
    unique_labels = np.unique(cluster_labels)
    
    # Initialize a list to store the first k clusters
    k_clusters = []
    
    # Loop over unique labels and extract the first k clusters (excluding noise)
    for label in unique_labels:
        if label != -1 and len(k_clusters) < k:  # Ignore noise (-1) and stop when we get k clusters
            cluster = X[cluster_labels == label]
            k_clusters.append(cluster)

    # Visualization of clusters using the first two features for 2D plotting
    plt.figure(figsize=(10, 6))
    
    # Loop over each cluster and plot
    for label in unique_labels:
        if label == -1:
            # Plot noise points in black
            plt.scatter(X[cluster_labels == label][:, 0], X[cluster_labels == label][:, 1], 
                        c='k', marker='x', label='Noise')
        else:
            if len(k_clusters) < k:  # Only plot up to k clusters
                # Generate a color for the cluster
                color = plt.cm.rainbow(float(label) / len(unique_labels))
                plt.scatter(X[cluster_labels == label][:, 0], X[cluster_labels == label][:, 1], 
                            c=[color], label=f'Cluster {label}')
    
    # Add labels and title to the plot
    plt.title(f'DBSCAN Clustering with First {k} Clusters')
    plt.legend()
    plt.show()
    
    return k_clusters


In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features.values

  
print(extract_and_visualize_k_clusters_with_dbscan(X, 3))


In [34]:
import warnings
import math
import copy
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import euclidean_distances
from imblearn.over_sampling.base import BaseOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.exceptions import raise_isinstance_error
from imblearn.utils import check_neighbors_object
from imblearn.utils.deprecation import deprecate_parameter